In [1]:
# !pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"

In [2]:
# !pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e
# !pip install -q -U git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)

2024-04-17 10:05:28.137374: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-17 10:05:28.173945: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 10:05:28.173974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 10:05:28.174831: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-17 10:05:28.180831: I tensorflow/core/platform/cpu_feature_guar

[2024-04-17 10:05:29,588] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


In [7]:
from datasets import load_dataset
def format_dolly(sample):
    context = f"\n[question]  {sample['instruction']} [/question]"
    instruction = f"<s> [INST] <<SYS>> Answer the question based on the context below. <</SYS>> \n[context]: {sample['context']} [/context]" if len(sample["context"]) > 0 else None
    response = f"[/INST] [answer] {sample['response']} [/answer]"
    # join all the parts together
    prompt = "".join([i for i in [instruction, context, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

# apply prompt template per sample
dataset = load_dataset("Vishwanath0912/qa_en_hi", split="train")

# Shuffle the dataset
dataset_shuffled = dataset.shuffle(seed=42)

# Select the first 50 rows from the shuffled dataset, comment if you want 15k
dataset = dataset_shuffled

train_data = dataset.map(template_dataset, remove_columns=list(dataset.features))
train_data

Map:   0%|          | 0/259 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 259
})

In [29]:
model_name = "meta-llama/Llama-2-7b-hf"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
output_dir="trained_weigths"

peft_config = LoraConfig(
        lora_alpha=16, 
        lora_dropout=0.05,
        r=64,
        bias="none",
        target_modules =["q_proj", "v_proj", "k_proj", "down_proj", "gate_proj", "up_proj"],
        task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=3,                       # number of training epochs
    per_device_train_batch_size=16,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=5e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)

trainer = SFTTrainer(
    model=merged_model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.634000,1.705342
1,1.523600,1.593814
2,1.466100,1.582825


TrainOutput(global_step=327, training_loss=1.589648884006232, metrics={'train_runtime': 3416.2173, 'train_samples_per_second': 12.294, 'train_steps_per_second': 0.096, 'total_flos': 1.7919495695499264e+17, 'train_loss': 1.589648884006232, 'epoch': 2.99})

In [34]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)

('trained_weigths/tokenizer_config.json',
 'trained_weigths/special_tokens_map.json',
 'trained_weigths/tokenizer.model',
 'trained_weigths/added_tokens.json',
 'trained_weigths/tokenizer.json')

In [6]:
from peft import AutoPeftModelForCausalLM

finetuned_model = "./trained_weigths/"
compute_dtype = getattr(torch, "float16")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

model = AutoPeftModelForCausalLM.from_pretrained(
     finetuned_model,
     torch_dtype=compute_dtype,
     return_dict=False,
     low_cpu_mem_usage=True,
     device_map=device,
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("./merged_model",safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained("./merged_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('./merged_model/tokenizer_config.json',
 './merged_model/special_tokens_map.json',
 './merged_model/tokenizer.model',
 './merged_model/added_tokens.json',
 './merged_model/tokenizer.json')

In [6]:
from datasets import load_dataset
def format_dolly(sample):
    context = f"\n[question]  {sample['instruction']} [/question]"
    instruction = f"<s> [INST] <<SYS>> Answer the question based on the context below. <</SYS>> \n[context]: {sample['context']} [/context]" if len(sample["context"]) > 0 else None
    response = f"[/INST]"
    # join all the parts together
    prompt = "".join([i for i in [instruction, context, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

# apply prompt template per sample
dataset = load_dataset("Vishwanath0912/qa_en_hi", split="validation")

# Shuffle the dataset
# dataset_shuffled = dataset.shuffle(seed=42)

# Select the first 50 rows from the shuffled dataset, comment if you want 15k
# dataset = dataset_shuffled

dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
dataset

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 54
})

In [7]:
def extract_answer(strings):
    answers = []
    for s in strings:
        start = s.find('[answer]') + len('[answer]')
        end = s.find('[/answer]')
        in_start = s.find('[/INST]')+len('[/INST]')
        if start != -1 and end != -1:
            answers.append(s[start:end])
        else:
            answers.append(s[in_start:])
            # print(s)
    return answers

In [8]:
y_pred=[]
for i in tqdm(range(len(dataset))):
    prompt = dataset[i]['text']
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    max_new_tokens = 32
    temperature = 0.1
    output = merged_model.generate(input_ids=inputs.input_ids, 
                               max_length=len(inputs.input_ids[0]) + max_new_tokens, 
                               temperature=temperature, 
                               pad_token_id=tokenizer.eos_token_id,
                               eos_token_id=tokenizer.pad_token_id)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    y_pred.append(generated_text)

100%|███████████████████████████████████████████| 54/54 [00:50<00:00,  1.07it/s]


In [9]:
fin_ans = extract_answer(y_pred)
file = f"meta18_llama_{max_new_tokens}_{temperature}.txt"
with open(file,'w') as f:
    for i in y_pred:
        f.write(i)
        f.write(f"\n")
import json

# Load the JSON file
with open('chat_gpt_3.5.json', 'r') as f:
    data = json.load(f)

# Get the keys
keys = list(data.keys())

print(keys)
data = dict(zip(keys, fin_ans))

# Write the dictionary to a JSON file
o_file = f"meta18_llama_{max_new_tokens}_{temperature}.json"
with open(o_file, 'w') as f:
    json.dump(data, f)
print("File output at ",o_file)

['4', '9', '19', '49', '50', '54', '58', '68', '72', '80', '88', '89', '90', '107', '115', '133', '137', '140', '166', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234']
File output at  meta18_llama_32_0.1.json
